# Analisis Exploratorio de Datos - Coursera

Importamos las librerias necesarias

In [2]:
#Importamos las librerias necesarias.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import ydata_profiling
from ydata_profiling import ProfileReport
import langdetect
from langdetect import detect
import googletrans
from googletrans import Translator

%matplotlib inline
%config IPCompleter.greedy=True

c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Importamos los Datasets
coursera_courses = pd.read_csv(r"C:\Users\miche\OneDrive\Escritorio\Henry\Proyecto_DatAnalitycs\Datasets_DB\coursera_v2.csv", na_values=" ") #dataset importado ok
coursera_data = pd.read_csv(r"C:\Users\miche\OneDrive\Escritorio\Henry\Proyecto_DatAnalitycs\DataSets_originales\coursea_data.csv", na_values=0)

Verificamos que los datasets fueron importados de forma correcta

In [4]:
coursera_courses.head(2)

,name,institution,course_url,course_id,reviews,reviewers,n_enrolled,date_reviews,review_year,review_month_name,review_day_name,n_reviews,mean_rating,rating,MOOC
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,This is an extremely basic course. Machine lea...,By Deleted A,1.0,2017-03-18,2017.0,Marzo,Sabado,12677.0,4.739923,1.0,Coursera
1,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,The course is ok but the certification procedu...,By Bruno C,1.0,2015-11-09,2015.0,Noviembre,Lunes,12677.0,4.739923,1.0,Coursera


In [5]:
coursera_data.head(2)

,Unnamed: 0,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,134.0,(ISC)² Systems Security Certified Practitioner...,(ISC)²,SPECIALIZATION,4.7,Beginner,5.3k
1,743.0,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,COURSE,4.7,Intermediate,17k


Borramos las columnas en el dataset "coursera_courses" que no aportaran valor a nuestro EDA para la realizacion del Dashboard

In [6]:
coursera_courses.drop(labels=["institution", "course_url", "reviews", "reviewers", "n_enrolled", "date_reviews", "review_year", "review_month_name","review_day_name"], axis=1, inplace=True)

Verificamos el resultado

In [7]:
coursera_courses.head()

,name,course_id,n_reviews,mean_rating,rating,MOOC
0,Machine Learning,machine-learning,12677.0,4.739923,1.0,Coursera
1,Machine Learning,machine-learning,12677.0,4.739923,1.0,Coursera
2,Machine Learning,machine-learning,12677.0,4.739923,1.0,Coursera
3,Machine Learning,machine-learning,12677.0,4.739923,1.0,Coursera
4,Machine Learning,machine-learning,12677.0,4.739923,1.0,Coursera


Borramos la columna rating, ya que tenemos el prometio de cada curso por lo que la misma ya no es necesaria

In [8]:
coursera_courses.drop(labels=["rating"], axis=1, inplace=True)

Verificamos el resultado

In [9]:
coursera_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519947 entries, 0 to 519946
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   name         519947 non-null  object 
 1   course_id    519947 non-null  object 
 2   n_reviews    519947 non-null  float64
 3   mean_rating  519947 non-null  float64
 4   MOOC         519947 non-null  object 
dtypes: float64(2), object(3)
memory usage: 19.8+ MB


Creamos una nueva variable para obtener el idioma, el cual estara definido por la interpretacion del lenguaje en el cual se hizo el titulo

In [10]:
coursera_courses.insert(4, "language", value="")


Vemos los valores unicos de nuestra nueva columna para poder realizar la transformacion

In [11]:
coursera_courses.language.value_counts()

    519947
Name: language, dtype: int64

Verificamos duplicados y procedemos a borrar por la columna "name"

In [12]:
#Verificamos la cantidad de duplicados
coursera_courses.duplicated().sum()

#Procedemos a borrarlos
coursera_courses.drop_duplicates(keep="first", inplace=True)

Ahora tenemos un dataset mas reducido para poder trabajar en nuestro dashboard

In [13]:
coursera_courses

,name,course_id,n_reviews,mean_rating,language,MOOC
0,Machine Learning,machine-learning,12677.0,4.739923,,Coursera
12677,Indigenous Canada,indigenous-canada,1460.0,4.816438,,Coursera
14137,The Science of Well-Being,the-science-of-well-being,8199.0,4.929504,,Coursera
22336,Technical Support Fundamentals,technical-support-fundamentals,12055.0,4.767482,,Coursera
34391,Become a CBRS Certified Professional Installer...,google-cbrs-cpi-training,33.0,4.818182,,Coursera
...,...,...,...,...,...,...
518574,Hacia una práctica constructivista en el aula,aulaconstructivista,569.0,4.905097,,Coursera
519143,Hypothesis-Driven Development,uva-darden-agile-testing,106.0,4.500000,,Coursera
519249,Accounting Data Analytics with Python,accounting-data-analytics-python,8.0,3.500000,,Coursera
519257,Introduction to Molecular Spectroscopy,spectroscopy,408.0,4.607843,,Coursera


Extraeremos el idioma de cada titulo para poder tener el idioma del curso.

In [14]:
#Transformar idiomas con detect.
def detect_language(idioma): # Definimos la función. 
    try: # Usamos try-except para que maneje los errores y pueda ejecutarse sin problemas. 
        return detect(idioma) # Aplicamos la función "detect" al parametro enviado.  
    except:
        return "Unknow" #retornamos nulo si nos encontramos con problemas. 

#Aplicacion de la funcion a las columnas language y subtitles
coursera_courses.loc[:, 'language'] = coursera_courses['name'].apply(detect_language) # Aplicamos todos los valores detectados en la función a la columna language

Reemplazamos los valores transformados por la funcion detect_language

In [15]:
#Cambiamos
coursera_courses.language = coursera_courses.language.replace({"en":"English", "es":"Spanish", "fr":"French", "zh-tw": "Chinese", "zh-cn": "Chinese", "pt":"Portuguese", "it":"Italian", "de":"Deutsch", "ar":"Arabic", "tl":"English", "ru":"Russian", "et":"Etiopian"})
coursera_courses.language = coursera_courses.language.replace({"ca":"English","no":"Norway","ja":"Japan", "nl":"Netherlands", "af":"Afghanistan"})
coursera_courses.language = coursera_courses.language.replace({"ro":"Rumania", "da":"Denmark", "sv": "Spanish", "tr":"Turkey"})
coursera_courses.language = coursera_courses.language.replace({"id":"Indonesia", "vi":"English", "sw":"Sweden","pl":"Poland", "th": "Thailand", "hr":"Croatia", "ko":"Japan", "so":"Somalia", "lt": "Lituania"})

Dejamos solos los "cursos" en certificaciones

In [16]:
coursera_data = coursera_data[coursera_data["course_Certificate_type"]=="COURSE"]

Borramos las columnas en el dataset "coursera_data" que no aportaran valor a nuestro EDA para la realizacion del Dashboard

In [17]:
coursera_data.drop(labels=["Unnamed: 0", "course_organization", "course_Certificate_type", "course_rating"], axis=1, inplace=True)

In [18]:
coursera_data

,course_title,course_difficulty,course_students_enrolled
1,A Crash Course in Causality: Inferring Causal...,Intermediate,17k
2,A Crash Course in Data Science,Mixed,130k
3,A Law Student's Toolkit,Mixed,91k
4,A Life of Happiness and Fulfillment,Mixed,320k
5,ADHD: Everyday Strategies for Elementary Students,Beginner,39k
...,...,...,...
880,Математика и Python для анализа данных,Beginner,67k
883,Основы программирования на Python,Beginner,83k
884,Основы разработки на C++: белый пояс,Intermediate,41k
885,Погружение в Python,Intermediate,45k


Unimos nuestros datasets

In [19]:
coursera_f = pd.merge(coursera_courses, coursera_data, left_on="name", right_on="course_title")

In [20]:
coursera_f

,name,course_id,n_reviews,mean_rating,language,MOOC,course_title,course_difficulty,course_students_enrolled
0,Machine Learning,machine-learning,12677.0,4.739923,English,Coursera,Machine Learning,Mixed,3.2m
1,Indigenous Canada,indigenous-canada,1460.0,4.816438,Indonesia,Coursera,Indigenous Canada,Mixed,51k
2,The Science of Well-Being,the-science-of-well-being,8199.0,4.929504,English,Coursera,The Science of Well-Being,Mixed,2.5m
3,Technical Support Fundamentals,technical-support-fundamentals,12055.0,4.767482,English,Coursera,Technical Support Fundamentals,Beginner,280k
4,Become a CBRS Certified Professional Installer...,google-cbrs-cpi-training,33.0,4.818182,English,Coursera,Become a CBRS Certified Professional Installer...,Mixed,210k
...,...,...,...,...,...,...,...,...,...
454,Geopolitics of Europe,geopolitics-europe,83.0,3.734940,English,Coursera,Geopolitics of Europe,Mixed,19k
455,Osteoarchaeology: The Truth in Our Bones,truthinourbones-osteoarchaeology-archaeology,124.0,4.580645,English,Coursera,Osteoarchaeology: The Truth in Our Bones,Beginner,22k
456,Mathematics for Machine Learning: PCA,pca-machine-learning,522.0,3.448276,English,Coursera,Mathematics for Machine Learning: PCA,Intermediate,33k
457,Математика и Python для анализа данных,mathematics-and-python,832.0,4.627404,Russian,Coursera,Математика и Python для анализа данных,Beginner,67k


In [21]:
coursera_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459 entries, 0 to 458
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      459 non-null    object 
 1   course_id                 459 non-null    object 
 2   n_reviews                 459 non-null    float64
 3   mean_rating               459 non-null    float64
 4   language                  459 non-null    object 
 5   MOOC                      459 non-null    object 
 6   course_title              459 non-null    object 
 7   course_difficulty         459 non-null    object 
 8   course_students_enrolled  459 non-null    object 
dtypes: float64(2), object(7)
memory usage: 35.9+ KB


In [22]:
coursera_f["course_students_enrolled"] = coursera_f["course_students_enrolled"].str.replace("k", "000")
coursera_f["course_students_enrolled"] = coursera_f["course_students_enrolled"].str.replace("m", "00000")
coursera_f["course_students_enrolled"] = coursera_f["course_students_enrolled"].str.replace(".", "")

C:\Users\miche\AppData\Local\Temp\ipykernel_2864\3556946328.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  coursera_f["course_students_enrolled"] = coursera_f["course_students_enrolled"].str.replace(".", "")


In [23]:
#Cambiamos el dtype
coursera_f["course_students_enrolled"] = coursera_f["course_students_enrolled"].astype(int)


In [24]:
coursera_f.head(2)

,name,course_id,n_reviews,mean_rating,language,MOOC,course_title,course_difficulty,course_students_enrolled
0,Machine Learning,machine-learning,12677.0,4.739923,English,Coursera,Machine Learning,Mixed,3200000
1,Indigenous Canada,indigenous-canada,1460.0,4.816438,Indonesia,Coursera,Indigenous Canada,Mixed,51000


Borramos las columnas en el dataset "coursera_courses" que no aportaran valor a nuestro EDA para la realizacion del Dashboard

In [25]:
coursera_f.drop(labels=["course_title"], axis=1, inplace=True)

In [26]:
coursera_f

,name,course_id,n_reviews,mean_rating,language,MOOC,course_difficulty,course_students_enrolled
0,Machine Learning,machine-learning,12677.0,4.739923,English,Coursera,Mixed,3200000
1,Indigenous Canada,indigenous-canada,1460.0,4.816438,Indonesia,Coursera,Mixed,51000
2,The Science of Well-Being,the-science-of-well-being,8199.0,4.929504,English,Coursera,Mixed,2500000
3,Technical Support Fundamentals,technical-support-fundamentals,12055.0,4.767482,English,Coursera,Beginner,280000
4,Become a CBRS Certified Professional Installer...,google-cbrs-cpi-training,33.0,4.818182,English,Coursera,Mixed,210000
...,...,...,...,...,...,...,...,...
454,Geopolitics of Europe,geopolitics-europe,83.0,3.734940,English,Coursera,Mixed,19000
455,Osteoarchaeology: The Truth in Our Bones,truthinourbones-osteoarchaeology-archaeology,124.0,4.580645,English,Coursera,Beginner,22000
456,Mathematics for Machine Learning: PCA,pca-machine-learning,522.0,3.448276,English,Coursera,Intermediate,33000
457,Математика и Python для анализа данных,mathematics-and-python,832.0,4.627404,Russian,Coursera,Beginner,67000


In [ ]:
#Exportamos
coursera_f.to_csv("coursera_EDA.csv", index=False)